In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 협업필터링 추천시스템 (투믈리에)

Model-based : 사용자와 아이템 간의 관계에서 잠재 요인을 찾아내는 "행렬 분해" 기법(SVD) 사용 

(1) **NEURAL MF** 
- 일반적인 MF에 딥러닝 적용한 모델
- 유저와 아이템 간 비선형적 관계 학습 및 추론

(2) **DEEP FM**
- FM + Deep Learning
- ccategorical, numerical feature -> embedding vector 변환
- 저차원, 고차원의 feature interactons 학습

(3) **DCN (Deep & Cross Network)** -> 사용
- feature interaction의 중요성을 기반으로 cross network 적용
- cross network 통해 feature 조합 형성, feature간 상호작용 정보 활용 가능
- 예측하고자 하는 Y는 feature의 독립적인 영향뿐 아니라 feature간 조합에서 발생하는 영향에도 의존한다고 가정함

(4) GCN
- 상품과 사용자 기반 그래프 추천시스템
- 상품과 사용자를 노드로, 인터랙션을 링크로써 그래프를 정의함 -> 노드 간 유사성 측정 -> 같은 구매 패턴을 보이는 사용자끼리 상품 추천 제공, 사용자가 좋아했던 상품과 유사한 또다른 상품을 추천



- 유저당 리뷰 개수가 3개 이상 : 572,954개 리뷰
- 유저당 리뷰 개수가 5개 이상 : 229,665개 리뷰

## DCN

참고 : https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/dcn.ipynb#scrollTo=ikhIvrku-i-L

https://www.tensorflow.org/recommenders/examples/dcn

**What is Deep & Cross Network (DCN)?** 

DCN was designed to learn explicit and bounded-degree cross features more effectively. It starts with an input layer (typically an embedding layer), followed by a *cross network* containing multiple cross layers that models explicit feature interactions, and then combines
with a *deep network* that models implicit feature interactions.


*   Cross Network. This is the core of DCN. It explicitly applies feature crossing at each layer, and the highest
polynomial degree increases with layer depth. The following figure shows the $(i+1)$-th cross layer.
<div class="fig figcenter fighighlight">
<center>
  <img src="http://drive.google.com/uc?export=view&id=1QvIDptMxixFNp6P4bBqMN4AYAhAIAYQZ" width="50%" style="display:block">
  </center>
</div>
*   Deep Network. It is a traditional feedforward multilayer perceptron (MLP).

The deep network and cross network are then combined to form DCN [[1](https://arxiv.org/pdf/2008.13535.pdf)]. Commonly, we could stack a deep network on top of the cross network (stacked structure); we could also place them in parallel (parallel structure). 


<div class="fig figcenter fighighlight">
<center>
  <img src="http://drive.google.com/uc?export=view&id=1WtDUCV6b-eetUnWVCAmcPh8mJFut5EUd" hspace="40" width="30%" style="margin: 0px 100px 0px 0px;">
  <img src="http://drive.google.com/uc?export=view&id=1xo_twKb847hasfss7JxF0UtFX_rEb4nt" width="20%">
  </center>
</div>

In [2]:
# 필요한 라이브러리 호출

import os
import sys
import gc
import glob
import joblib
from tqdm import tqdm
import pprint

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import pandas as pd

!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

import keras
import tensorflow as tf
import tensorflow_recommenders as tfrs

from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, Callback
from tensorflow.keras.utils import plot_model

from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
import tensorflow_datasets as tfds

     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 4.2 MB 25.4 MB/s 


### Data processing

In [13]:
# 데이터 불러오기 & 피처 타입 확인

# 데이터 불러오기
df = pd.read_csv("/content/drive/Shareddrives/컨퍼런스/clus_pororo_final.csv", encoding="utf-8")

# 리뷰 3개 이상 유저 수
df3 = df['user'].value_counts().reset_index()
print('총 유저 수:', df['user'].nunique())
print('3개 이상 유저 수:', len(df3.query('user >= 3')))

# 최종 사용할 데이터셋
my_user = df3.query('user >= 3')['index'].tolist()
isin_con = df['user'].isin(my_user)
df = df.loc[isin_con].reset_index().iloc[:, 1:]

# rating_per_user 변수 생성
# 유저별 평균 평점 
df4 = df.groupby('user')['total_score'].mean().reset_index()
df = pd.merge(left=df, right=df4, on='user', how='left')
df = df.rename(columns={'total_score_x': 'total_score', 'total_score_y': 'rating_per_user'})

# like 변수 생성 
# 가구에 대한 유저의 평점 > 해당 유저의 평균 평점 -> 1
# 가구에 대한 유저의 평점 < 해당 유저의 평균 평점 -> 0
#def mylike(df):
#    if df['total_score'] >= df['rating_per_user']:
#        return 1
#    else:
#        return 0

#df['like'] = df.apply(mylike, axis=1)

# like 분포 확인
#df['like'].value_counts()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


총 유저 수: 660334
3개 이상 유저 수: 139608


In [4]:
# 결측치 확인

df.isnull().sum()

user                   0
date                   0
total_score            0
dur_score          12274
price_score        12276
design_score       12275
delivery_score     12276
options                0
review                 0
help_count             0
item_name              0
item_brand             0
item_sale_price        0
item_category1         0
item_category2         0
item_category3         0
item_score             0
item_count             0
review2                0
n1                     0
n2                     0
n3                     0
n4                     0
rating_per_user        0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572935 entries, 0 to 572934
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user             572935 non-null  object 
 1   date             572935 non-null  object 
 2   total_score      572935 non-null  float64
 3   dur_score        560661 non-null  float64
 4   price_score      560659 non-null  float64
 5   design_score     560660 non-null  float64
 6   delivery_score   560659 non-null  float64
 7   options          572935 non-null  object 
 8   review           572935 non-null  object 
 9   help_count       572935 non-null  object 
 10  item_name        572935 non-null  object 
 11  item_brand       572935 non-null  object 
 12  item_sale_price  572935 non-null  int64  
 13  item_category1   572935 non-null  object 
 14  item_category2   572935 non-null  object 
 15  item_category3   572935 non-null  object 
 16  item_score       572935 non-null  floa

In [14]:
str_features = ["user", "date", "options", "review2", "item_name", "item_brand", 
                "item_category1", "item_category2", "item_category3"]

int_features = ["total_score", #"dur_score", "price_score", 	"design_score", "delivery_score", -> 일단 결측이 많아서 제외하고 진행
                "help_count", "item_sale_price", 	"item_score", "item_count", "n1", "n2", "n3", "n4", "rating_per_user"] # 감성점수 추가

# y -> "rating_per_user"

feature_names = str_features + int_features

### Model construction

In [15]:
def settype(df) :
  
  # help count: str -> int
  def preprocess_helpcount(x) :
    return str(x).replace(',', '')
    
  df['help_count'] = df['help_count'].apply(preprocess_helpcount).astype(int)

  # float -> int을 위한 전처리
  df['total_score'] = df['total_score'].apply(lambda x : x * 10)
  df['item_score'] = df['item_score'].apply(lambda x : x * 10)
  df['n1'] = df['n1'].apply(lambda x : x * 10) 
  df['n2'] = df['n2'].apply(lambda x : x * 10)
  df['n3'] = df['n3'].apply(lambda x : x * 10)
  df['n4'] = df['n4'].apply(lambda x : x * 10)
  df['rating_per_user'] = df['rating_per_user'].apply(lambda x : x * 10)

  df['total_score'] = np.asarray(df['total_score']).astype(np.int)
  df['item_score'] = np.asarray(df['item_score']).astype(np.int)
  df['n1'] = np.asarray(df['n1']).astype(np.int)
  df['n2'] = np.asarray(df['n2']).astype(np.int)
  df['n3'] = np.asarray(df['n3']).astype(np.int)
  df['n4'] = np.asarray(df['n4']).astype(np.int)
  df['rating_per_user'] = np.asarray(df['rating_per_user']).astype(np.int)

  for f in str_features:
    if df[f].dtypes == float:
      df[f] = np.asarray(df[f]).astype(np.int)

  for f in int_features:
    df[f] = np.asarray(df[f]).astype(np.int)
            
  return df

In [16]:
settype(df)

,user,date,total_score,dur_score,price_score,design_score,delivery_score,options,review,help_count,item_name,item_brand,item_sale_price,item_category1,item_category2,item_category3,item_score,item_count,review2,n1,n2,n3,n4,rating_per_user
0,GuYung,2021.12.23 ∙ 오늘의집 구매,50,5.0,5.0,5.0,5.0,상품명: A사이드테이블 / 색상: 화이트,깔끔하고 예뻐서 마음에 들어요!!!!,2,당일출고 순수원목 A사이드테이블 3colors,먼데이하우스,18900,테이블/책상,사이드테이블,사이드테이블,46,36825,깔끔하고 예뻐서 마음에 들어요!!!!,34,34,34,34,50
1,딘뇨잉,2021.12.29 ∙ 오늘의집 구매,50,5.0,5.0,5.0,5.0,상품명: A사이드테이블 / 색상: 화이트,좋아요 레이스천이랑해서 잘사용중임ㅎㅎ,1,당일출고 순수원목 A사이드테이블 3colors,먼데이하우스,18900,테이블/책상,사이드테이블,사이드테이블,46,36825,좋아요 레이스 천이랑 해서 잘사용중임ㅎㅎ,34,34,34,34,50
2,토순이언니2,2021.12.29 ∙ 오늘의집 구매,50,5.0,5.0,5.0,5.0,상품명: A사이드테이블 / 색상: 화이트,말해뭐해>< 역시 제선택은 탁월했어요 너무예쁘고 조립도쉬워요 상상한그대로에요,1,당일출고 순수원목 A사이드테이블 3colors,먼데이하우스,18900,테이블/책상,사이드테이블,사이드테이블,46,36825,말해 뭐 해>< 역시 지 선택은 탁월했어요 너무 예쁘고 조립도 쉬워요 상상한 그대로에요,34,39,34,34,50
3,블링블링★,2021.12.28 ∙ 오늘의집 구매,50,4.0,4.0,4.0,4.0,상품명: A사이드테이블 / 색상: 우드,아이방에 놓고 이것저것 올려놓기 좋아요 배송 빨라요,1,당일출고 순수원목 A사이드테이블 3colors,먼데이하우스,18900,테이블/책상,사이드테이블,사이드테이블,46,36825,아이 방에 놓고 이것저것 올려놓기 좋아요 배송 빨라요,34,34,36,34,47
4,쿠크돳ㅡ,2021.12.28 ∙ 오늘의집 구매,50,5.0,5.0,5.0,5.0,상품명: A사이드테이블 / 색상: 우드,인테리어로 진짜 쵝오에요! 다음에 이사갈때도 재구매 무조건입니더,1,당일출고 순수원목 A사이드테이블 3colors,먼데이하우스,18900,테이블/책상,사이드테이블,사이드테이블,46,36825,인테리어로 진짜 쵝오에요! 다음에 이사 갈 때도 재구매 무조건입니다,48,34,34,34,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572930,박소선93,2020.08.01 ∙ 오늘의집 구매,43,3.0,4.0,5.0,5.0,화이트,비가 많이왔는데도 기사님이 하나도 안젖게 가지고와주시고 되게 친절하셨어요!! 내구성...,0,테온 에어프라이기 멀티 수납 800 렌지대,유에라가구,95000,서랍/수납장,주방수납장,렌지대,46,10,비가 많이 왔는데도 기사님이 하나도 안 젖게 가지고 와주시고 되게 친절하셨어요!! ...,34,32,26,34,38
572931,우석9214,2021.09.04 ∙ 오늘의집 구매,40,4.0,4.0,4.0,4.0,02_크림 DF638276,수납이야 많을수록좋아요 잡다한 식품이나 주방기구를 수낭했어요,0,드림 1800/주방수납장 (양문장) 2colors,동서가구,119000,서랍/수납장,주방수납장,키큰장,45,10,수납이야 많을수록좋아요 잡다한 식품이나 주방기구를 수납했어요,34,37,34,34,45
572932,똥그니집고치자,2021.03.05 ∙ 오늘의집 구매,50,5.0,5.0,5.0,5.0,01_그레이 DF638276,아담하고 좋아요 ㅎ 깊어서 이것저것 넣기도 편하구요 ㅎ 냄새나서 처음에는 문열어놓고...,0,드림 1800/주방수납장 (양문장) 2colors,동서가구,119000,서랍/수납장,주방수납장,키큰장,45,10,아담하고 좋아요 ㅎ 깊어서 이것저것 넣기도 편하고요 ㅎ 냄새나다 처음에는 문 열어놓...,34,34,34,34,50
572933,매력란,2021.02.03 ∙ 오늘의집 구매,50,5.0,5.0,5.0,5.0,01_그레이 DF638276,베란다에두고 그릇장으로 사용중인데 수납많이되서 좋아요,0,드림 1800/주방수납장 (양문장) 2colors,동서가구,119000,서랍/수납장,주방수납장,키큰장,45,10,베란다에 두고 그릇장으로 사용 중인데 수납 많이 돼서 좋아요,34,36,34,34,50


In [17]:
# *train/test 분리 
print(len(df))
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
train = df[:458348] # 80%
test = df[458348:]

572935


In [18]:
add_train = train[feature_names]
add_test = test[feature_names]

add_train

,user,date,options,review2,item_name,item_brand,item_category1,item_category2,item_category3,total_score,help_count,item_sale_price,item_score,item_count,n1,n2,n3,n4,rating_per_user
0,다용잉잉잉,2018.12.16 ∙ 오늘의집 구매,네츄럴,가성비 좋아요! 배송도 주문하고 다음날 바로 배송 처리해주셨어요,[5%쿠폰] 벤트우드 스툴 5colors,로제까사,의자,스툴/벤치,스툴/벤치,45,0,9800,47,5777,34,34,46,34,47
1,Merry0000,2019.08.25 ∙ 오늘의집 구매,D테이블 우드,다른 분 리뷰에서 본 대로 생각보다 아주 작진 않고 넉넉한 편이에요! 혼자서 넉넉하...,당일출고 순수원목 D테이블 접이식 거실테이블 2colors,먼데이하우스,테이블/책상,거실/소파테이블,거실/소파테이블,50,0,40900,46,5082,34,34,34,34,50
2,moaco,2021.02.10 ∙ 오늘의집 구매,1000책상/ 화이트오크,"업무용 책상 서브용으로 하나 구매했습니다. 디자인, 내구성 모두 만족합니다. 배송도...",아이언 1000 1인용 철제 컴퓨터 책상 6colors,상일리베가구,테이블/책상,책상,일반책상,50,0,58900,44,1097,34,41,40,34,50
3,다람쥐뽀쪼,2020.05.18 ∙ 오늘의집 구매,선택: 로맨틱 Q_프레임 / 색상: 내추럴,이쁘고 좋아요~~그런데 밑에 다리 네 개 제외하고 중앙에 두 개밖에 없어서 조금 불...,[10%쿠폰] 로맨틱 스칸딕 원목침대 프레임,레이디가구,침대,침대프레임,일반침대,40,0,189000,46,1222,34,29,34,34,40
4,Joo9016,2021.12.25 ∙ 오늘의집 구매,제품선택: 슈퍼싱글SS_프레임만 / 색상선택: 화이트,큰 캐리어 들어가는 줄 알고 시켰는데 들어가진 않네요 ㅠ 그 점 조금 아쉽고 수납공...,헤르만 4단 멀티 통서랍형 수납침대 SS/Q,리샘,침대,침대프레임,수납침대,30,0,159000,46,2142,37,34,34,34,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458343,곰보와소보로,2021.05.31 ∙ 오늘의집 구매,팬지 테이블 L (화이트),여기가 제일 싸고 튼튼해 보여서 샀어요 아주 잘 사용하고 있습니다,팬지 원목 좌식 거실테이블,리빙힙,테이블/책상,거실/소파테이블,거실/소파테이블,50,0,31900,47,1101,34,38,34,34,45
458344,쓰쥬,2019.06.20 ∙ 오늘의집 구매,네추럴ⓘP819535ⓟ,빠른 배송이었어서 너무 만족하고 다만 제가 멍청한 건지 조립하느라 애먹은 부분도 있...,심플 H형 책상 7colors,블루밍홈,테이블/책상,책상,일반책상,50,0,35000,46,1264,34,36,38,34,48
458345,윤현재2,2020.04.17 ∙ 오늘의집 구매,상품명: 토실토실 패브릭의자 / 색상: 그레이,의자 그레이 색 너무 예쁘고 푹신하고 좋아요,스툴 원목다리 패브릭 화장대 의자 (일반형) 3colors,그린우드,의자,스툴/벤치,스툴/벤치,43,0,13500,47,2664,34,37,34,34,42
458346,afaghhh,2020.08.11 ∙ 오늘의집 구매,대나무+유리 202.471.28,지금까지 산 거중에 제일 맘에 들어요 가격도 합리적이고 가성비 좋은데 조립도 쉬워요...,NESNA 네스나 원목 사이드테이블,이케아,서랍/수납장,협탁,협탁,48,0,20900,48,1519,34,37,34,42,44


In [19]:
str_features = ["user", "date", "options", "review2", "item_name", "item_brand", 
                "item_category1", "item_category2", "item_category3"]

int_features = ["total_score", "help_count", "item_sale_price", 	"item_score", "item_count", "n1", "n2", "n3", "n4"]

In [20]:
def DCN(df, str_features, int_features, df_type = 'train'):

    feature_names = str_features + int_features
    
    # 데이터 dict로 변환
    def generateDict(df):
        # str features는 encoding
        train_str_dict = {
            str_feature: [str(val).encode() for val in df[str_feature].values]
            for str_feature in str_features
            }
        # int features는 int
        train_int_dict = {
            int_feature: df[int_feature].values
            for int_feature in int_features
            }

        # label columns이 있다면~
#        try:
#            train_label_dict = {
#                'like' : df['like'].values
#            }
#            train_str_dict.update(train_label_dict)
#        except:
#            pass

        try:
            train_label_dict = {
                'rating_per_user' : df['rating_per_user'].values
            }
            train_str_dict.update(train_label_dict)
        except:
            pass
    
        train_str_dict.update(train_int_dict)
        
        return train_str_dict

    input_dict = generateDict(df)

    # tensor
    tensor = tf.data.Dataset.from_tensor_slices(input_dict)
    cached = tensor.shuffle(100_000).batch(8192).cache()
    # unique data 저장
    # train data 일 때, 
    if df_type == 'train':
        vocabularies = {}
    
        for feature_name in tqdm(feature_names):
            vocab = tensor.batch(1_000_000).map(lambda x: x[feature_name])
            vocabularies[feature_name] = np.unique(np.concatenate(list(vocab)))
    
        return cached, vocabularies
      
      # test data 일 때, 
    else:
        return cached

In [21]:
#!pip install --upgrade pandas
cached_train, vocabularies = DCN(add_train, str_features, int_features, df_type = 'train')
cached_test = DCN(add_test, str_features, int_features, df_type = 'test')

100%|██████████| 18/18 [02:04<00:00,  6.91s/it]


In [22]:
class model(tfrs.Model):
    
    def __init__(self, cross_layer_sizes, deep_layer_sizes, learning_rate, str_features, int_features, vocabularies, projection_dim = None, metric = 'binary'):
        super().__init__()
    
        self.embedding_dimension = 64
    
        self._all_features = str_features + int_features
        self._embeddings = {}
    
        # Compute embeddings for string features.
        for feature_name in str_features:
            vocabulary = vocabularies[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential(
                [tf.keras.layers.experimental.preprocessing.StringLookup(
                    vocabulary=vocabulary, mask_token=None),
                    tf.keras.layers.Embedding(len(vocabulary) + 1,
                    self.embedding_dimension)
                                             ])
          
    
        # Compute embeddings for int features.
        for feature_name in int_features:
            vocabulary = vocabularies[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential(
                [tf.keras.layers.experimental.preprocessing.IntegerLookup(
                    vocabulary=vocabulary, mask_value=None),
                    tf.keras.layers.Embedding(len(vocabulary) + 1,
                    self.embedding_dimension)
                                         ])
    
#         if use_cross_layer:
#             self._cross_layer = tfrs.layers.dcn.Cross(
#                 projection_dim = projection_dim,
#                 kernel_initializer = "glorot_uniform")
#         else:
#             self._cross_layer = None
        
        # Cross layer
        if cross_layer_sizes:
            self._cross_layer = [tfrs.layers.dcn.Cross(
                projection_dim = projection_dim,
                kernel_initializer = "glorot_uniform") for _ in range(cross_layer_sizes)]
        else:
            self._cross_layer = None
            
        # Deep layer
        self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu")
            for layer_size in deep_layer_sizes]
        
        # Output layer
        self._logit_layer = tf.keras.layers.Dense(1,
                                                  activation = 'sigmoid'
                                                  )

        # Metric
        if metric == 'binary':
            self.task = tfrs.tasks.Ranking(
            loss = tf.keras.losses.BinaryCrossentropy(),
            metrics=[
                    tf.keras.metrics.BinaryAccuracy(
                        name='binary_accuracy', dtype = None, threshold = 0.5)
                    ])
                    
        elif metric == 'reg':
            self.task = tfrs.tasks.Ranking(
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError("RMSE")
                    ])
        else:
            print('metric ERROR!')
            sys.exit(1)
    
    def call(self, features):
        # Concatenate embeddings
        embeddings = []
        for feature_name in self._all_features:
            embedding_fn = self._embeddings[feature_name]
            embeddings.append(embedding_fn(features[feature_name]))
    
        x = tf.concat(embeddings, axis=1)
    
        # Build Cross Network
#         if self._cross_layer is not None:
#             x = self._cross_layer(x)
        for cross_layer in self._cross_layer:
            x = cross_layer(x)
        
        # Build Deep Network
        for deep_layer in self._deep_layers:
            x = deep_layer(x)
    
        return self._logit_layer(x)
    
    def compute_loss(self, features, training=False, metric = 'reg'):
        labels = features.pop("rating_per_user")
        scores = self(features)
    
        return self.task(
            labels=labels,
            predictions=scores,
            )

In [23]:
learning_rate = 0.001
epochs = 20

DCNmodel = model(cross_layer_sizes = 2,
                  deep_layer_sizes = [512, 256, 128, 64],
                  learning_rate = learning_rate,
                  str_features = str_features,
                  int_features = int_features,
                  vocabularies = vocabularies,
                  projection_dim = None,
                  metric = 'reg'
                  )

DCNmodel.compile(optimizer = tf.keras.optimizers.Adam(learning_rate))

### Model training

In [21]:
from keras.callbacks import EarlyStopping
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 20)

In [26]:
history = DCNmodel.fit(cached_train,  epochs=20,  validation_split=0.15, verbose=True, callbacks=[callback])

ValueError: ignored

In [23]:
def run_models(cross_layer_sizes, deep_layer_sizes, projection_dim=None, num_runs=5) :
  models = []
  rmses = []

  for i in range(num_runs):
    DCNmodel = model(cross_layer_sizes = cross_layer_sizes,
                deep_layer_sizes = deep_layer_sizes,
                learning_rate = 0.001,
                str_features = str_features,
                int_features = int_features,
                vocabularies = vocabularies,
                projection_dim = None,
                metric = "reg")
    DCNmodel.compile(optimizer=tf.keras.optimizers.Adam(0.001))
    models.append(DCNmodel)

    DCNmodel.fit(cached_train, epochs=20, verbose=False)
    metrics = DCNmodel.evaluate(cached_test, return_dict=True)
    rmses.append(metrics["RMSE"])

  mean, stdv = np.average(rmses), np.std(rmses)

  return {"model": models, "mean": mean, "stdv": stdv}

In [ ]:
run_models(cross_layer_sizes =2, deep_layer_sizes=[512,256,128,64], projection_dim=None, num_runs=5) # 왜 안 돌아가지..

14/14 [==============================] - 12s 844ms/step - RMSE: 46.1509 - loss: 2129.7001 - regularization_loss: 0.0000e+00 - total_loss: 2129.7001


In [ ]:
print("DCN            RMSE mean: {:.4f}, stdv: {:.4f}".format(
    dcn_result["mean"], dcn_result["stdv"]))
print("DCN (low-rank) RMSE mean: {:.4f}, stdv: {:.4f}".format(
    dcn_lr_result["mean"], dcn_lr_result["stdv"]))
print("DNN            RMSE mean: {:.4f}, stdv: {:.4f}".format(
    dnn_result["mean"], dnn_result["stdv"]))

### Model understanding (피처 간 상호작용 확인)

The weight matrix $W$ in DCN reveals what feature crosses the model has learned to be important.

In [ ]:
model = dcn_result["model"][0]
mat = model._cross_layer._dense.kernel
features = model._all_features

block_norm = np.ones([len(features), len(features)])

dim = model.embedding_dimension

# Compute the norms of the blocks.
for i in range(len(features)):
  for j in range(len(features)):
    block = mat[i * dim:(i + 1) * dim,
                j * dim:(j + 1) * dim]
    block_norm[i,j] = np.linalg.norm(block, ord="fro")

plt.figure(figsize=(9,9))
im = plt.matshow(block_norm, cmap=plt.cm.Blues)
ax = plt.gca()
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
cax.tick_params(labelsize=10) 
_ = ax.set_xticklabels([""] + features, rotation=45, ha="left", fontsize=10)
_ = ax.set_yticklabels([""] + features, fontsize=10)